## Normalizar Columnas de la Agenda

In [29]:
import pandas as pd
import numpy as np
import datetime
import re
from datetime import timedelta
from datetime import date

In [23]:
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/00. Matrices BDD/02. Agendamiento\Repositorio 2 Años/AgendaSegmentacionCiclos.xlsx"

nombre_hoja = 'PrimerCiclo'

# Lee el archivo de Excel en un DataFrame de pandas
df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

## Normalizar la Data (Agenda)

In [11]:
nuevos_nombres = ["fecha_reserva","marca","trabajo_agenda","patente","box","trabajo","idr","detalle","submodelo","teleoperador_7d","oficina","fecha_estado","canal","estado","vigencia","teleoperador_agendo","rut_cliente","nombre_cliente","fono_casa_cliente","fono_oficina_cliente","fono_celular_cliente","mail_cliente","vin","oficina_idr","patente_idr","ciclo"]
df.columns = nuevos_nombres

In [12]:
df = df.astype(str)
df["fecha_reserva"] = pd.to_datetime(df["fecha_reserva"], format="%Y-%m-%d")
df["fecha_estado"] = pd.to_datetime(df["fecha_estado"], format="%Y-%m-%d")

In [13]:
df['vin'] = df['vin'].str.replace('O', '0')

In [14]:
df['vin'] = df['vin'].astype(str)

def clean_vin(vin):
    if len(vin) != 17:
        return ''
    return vin

df['vin'] = df['vin'].apply(clean_vin)

In [15]:
df['patente'] = df['patente'].astype(str)

def clean_patente(patente):
    if len(patente) != 7:
        return ''
    return patente

df['patente'] = df['patente'].apply(clean_patente)

In [16]:
df.replace("", np.nan, inplace=True)
#df['vin'] = df.groupby('patente')['vin'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [17]:
##ESTE ES
#df.replace("", np.nan, inplace=True)
df['vin'] = df.sort_values('fecha_reserva', ascending=False).groupby('patente')['vin'].transform('first')

In [18]:
df['vin_patente'] = np.where(df['vin'] == "", df['patente'], df['vin'])

In [19]:
df['oficina'] = df['oficina'].str.upper()
def asignar_unidad_negocios(oficina):
    if "NISSAN" in oficina:
        return "NISSAN"
    elif "TOYOTA" in oficina:
        return "TOYOTA"
    elif "DYP" in oficina:
        return "DYP"
    else:
        return "CHEVROLET"

df["unidad_negocios"] = df["oficina"].apply(asignar_unidad_negocios)

In [20]:

# Convertimos la columna 'fecha_reserva' a tipo datetime
df['fecha_reserva'] = pd.to_datetime(df['fecha_reserva'], format='%Y-%m-%d')

# Calculamos la fecha actual
today = pd.Timestamp.today()

# Agregamos una nueva columna 'ciclo'
df['ciclo'] = pd.cut(
    x=(today - df['fecha_reserva']).dt.days / 30,  # Calculamos la diferencia en meses
    bins=[-float('inf'), 0, 9, 18, 27, float('inf')],  # Definimos los límites de cada ciclo
    labels=['Ciclo 0', 'Primer Ciclo', 'Segundo Ciclo', 'Tercer Ciclo', 'Cuarto Ciclo']  # Definimos las etiquetas de cada ciclo
)

# Agregamos una condición para los casos en que la fecha de reserva es posterior a la fecha actual
df.loc[df['fecha_reserva'] > today, 'ciclo'] = 'Ciclo 0'


In [21]:
df.sort_values("fecha_reserva", ascending=False, inplace=True)
df_ultimoEstado = df.groupby("vin_patente").first()
df_ultimoEstado = df_ultimoEstado.astype(str)
df_ultimoEstado["fecha_reserva"] = pd.to_datetime(df_ultimoEstado["fecha_reserva"], format="%Y-%m-%d")
df_ultimoEstado["fecha_estado"] = pd.to_datetime(df_ultimoEstado["fecha_estado"], format="%Y-%m-%d")
df_ultimoEstado.to_excel("AgendaCortaUltimoEstado.xlsx", index=False)
df.to_excel("AgendaCortaALL.xlsx", index=False)

In [22]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

def to_excel_with_date_formatting(df, wb, sheet_name, date_format="yyyy-mm-dd", index=False):
    ws = wb.create_sheet(sheet_name)

    rows = dataframe_to_rows(df, index=index, header=True)
    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
            _ = ws.cell(row=r_idx, column=c_idx, value=value)

            if isinstance(value, pd.Timestamp):
                ws.cell(row=r_idx, column=c_idx).number_format = date_format

    if not index:
        ws.column_dimensions['A'].visible = False

        
def clean_sheet_name(sheet_name):
    invalid_chars = '[]:*?/\\'
    for char in invalid_chars:
        sheet_name = sheet_name.replace(char, '')
    return sheet_name[:31]  # El límite de caracteres para el nombre de una hoja en Excel es de 31


df_filtered = df_ultimoEstado[
    (df_ultimoEstado["trabajo_agenda"].isin(["PREVENTIVO", "CORRECTIVO", "GARANTIA"])) &
    (df_ultimoEstado["estado"].isin(["FACTURADO", "ENTREGADO"]))
]


for unidad in df_filtered["unidad_negocios"].unique():
    df_unidad = df_filtered[df_filtered["unidad_negocios"] == unidad]

    wb = openpyxl.Workbook()
    wb.remove(wb.active)  # Elimina la hoja predeterminada
    sheet_names = set()



    for oficina in df_unidad["oficina"].unique():
        df_oficina = df_unidad[df_unidad["oficina"] == oficina]

        sheet_name = f"{oficina}"
        cleaned_sheet_name = clean_sheet_name(sheet_name)

        counter = 1
        unique_sheet_name = cleaned_sheet_name
        while unique_sheet_name in sheet_names:
            unique_sheet_name = f"{cleaned_sheet_name}_{counter}"
            counter += 1

        sheet_names.add(unique_sheet_name)

        # Ordenar el DataFrame y crear una nueva copia
        df_oficina = df_oficina.sort_values(by="fecha_reserva", ascending=False)

        # Utilice la función personalizada para guardar el DataFrame con formato de fecha
        to_excel_with_date_formatting(df_oficina, wb, unique_sheet_name, date_format="dd-mm-yyyy", index=False)

    # Guardar el archivo de Excel
    wb.save(f"AgendaCortaUltimoEstado_{unidad}.xlsx")
